In [1]:
# the following codes are the answer of Q1
from collections import defaultdict, Counter
import re

# 清理文本
corpus = "Mamma mia! Mamma made marmalade at a mama market. Many marmots made amazing marmalade for mamma and mama mia"
corpus = corpus.lower()
corpus = re.sub(r"[^\w\s]", "", corpus)  # 去除标点符号
words = corpus.split()

# 每个单词拆成字符，并加上 </w>
tokenized_words = [list(word) + ['</w>'] for word in words]

# 定义函数统计符号对频率
def get_pair_freq(words):
    pairs = defaultdict(int)
    for word in words:
        for i in range(len(word)-1):
            pair = (word[i], word[i+1])
            pairs[pair] += 1
    return pairs

# BPE 迭代合并
K = 5
merges = []

for i in range(K):
    pairs = get_pair_freq(tokenized_words)
    if not pairs:
        break
    # 找频率最高的符号对
    best_pair = max(pairs, key=pairs.get)
    freq = pairs[best_pair]
    merges.append((best_pair, freq))
    
    # 执行合并
    new_words = []
    for word in tokenized_words:
        new_word = []
        j = 0
        while j < len(word):
            # 如果匹配符号对就合并
            if j < len(word) - 1 and (word[j], word[j+1]) == best_pair:
                new_word.append(''.join(best_pair))
                j += 2
            else:
                new_word.append(word[j])
                j += 1
        new_words.append(new_word)
    tokenized_words = new_words

# 输出结果
print("Selected merges and their frequencies:")
for i, (pair, freq) in enumerate(merges, 1):
    print(f"{i}: {pair} -> {freq}")

# 将最终符号用 {} 包起来
final_tokenization = []
for word in tokenized_words:
    final_tokenization.append(' '.join([f"{{{c}}}" if len(c) > 1 else c for c in word]))

print("\nFinal tokenization of corpus:")
print(' '.join(final_tokenization))

Selected merges and their frequencies:
1: ('m', 'a') -> 20
2: ('ma', '</w>') -> 5
3: ('d', 'e') -> 4
4: ('de', '</w>') -> 4
5: ('ma', 'r') -> 4

Final tokenization of corpus:
{ma} m {ma</w>} m i a {</w>} {ma} m {ma</w>} {ma} {de</w>} {mar} {ma} l a {de</w>} a t {</w>} a {</w>} {ma} {ma</w>} {mar} k e t {</w>} {ma} n y {</w>} {mar} m o t s {</w>} {ma} {de</w>} a {ma} z i n g {</w>} {mar} {ma} l a {de</w>} f o r {</w>} {ma} m {ma</w>} a n d {</w>} {ma} {ma</w>} m i a {</w>}


In [3]:
# the following codes are the answer of Q2
from collections import defaultdict

# 文法定义
grammar = {
    'S': [['NP', 'VP']],
    'VP': [['V', 'NP'], ['V']],
    'NP': [['Det', 'N']],
    'Det': [['the'], ['a']],
    'N': [['flies'], ['banana']],
    'V': [['flies'], ['like']]
}

# 将右侧映射到左侧，方便 CKY 查找
rhs_to_lhs = defaultdict(list)
for lhs, rules in grammar.items():
    for rule in rules:
        rhs_to_lhs[tuple(rule)].append(lhs)

# 句子
sentence = "the flies like a banana".split()
n = len(sentence)

# 初始化 CKY 表
table = [[set() for j in range(n+1)] for i in range(n)]

# 填充长度为1的 span
for i, word in enumerate(sentence):
    if (word,) in rhs_to_lhs:
        table[i][i+1].update(rhs_to_lhs[(word,)])

# 4CKY 迭代填表
for span in range(2, n+1):  # span 长度
    for i in range(n - span + 1):
        j = i + span
        for k in range(i+1, j):
            for B in table[i][k]:
                for C in table[k][j]:
                    if (B,C) in rhs_to_lhs:
                        table[i][j].update(rhs_to_lhs[(B,C)])

# 输出每个 cell
for i in range(n):
    for j in range(i+1, n+1):
        span_words = " ".join(sentence[i:j])
        print(f"({i},{j}): {table[i][j]} | span='{span_words}'")


(0,1): {'Det'} | span='the'
(0,2): {'NP'} | span='the flies'
(0,3): set() | span='the flies like'
(0,4): set() | span='the flies like a'
(0,5): {'S'} | span='the flies like a banana'
(1,2): {'V', 'N'} | span='flies'
(1,3): set() | span='flies like'
(1,4): set() | span='flies like a'
(1,5): set() | span='flies like a banana'
(2,3): {'V'} | span='like'
(2,4): set() | span='like a'
(2,5): {'VP'} | span='like a banana'
(3,4): {'Det'} | span='a'
(3,5): {'NP'} | span='a banana'
(4,5): {'N'} | span='banana'


In [11]:
# the following codes are the answer of Q3.a
# Tokenize the corpus
import re

corpus = [
    "The fruit flies like a swift arrow.",
    "Time flies like a bird; fruit flies like a banana.",
    "I like slp and math.",
    "I like pizza and pasta.",
    "Mamma made marmalade.",
    "The quick fox likes fruit."
]

tokenized_corpus = []
for sentence in corpus:
    tokens = re.findall(r"[a-zA-Z]+", sentence.lower())
    tokenized_corpus.append(tokens)

for i, tokens in enumerate(tokenized_corpus, 1):
    print(f"Sentence {i}: {tokens}")

Sentence 1: ['the', 'fruit', 'flies', 'like', 'a', 'swift', 'arrow']
Sentence 2: ['time', 'flies', 'like', 'a', 'bird', 'fruit', 'flies', 'like', 'a', 'banana']
Sentence 3: ['i', 'like', 'slp', 'and', 'math']
Sentence 4: ['i', 'like', 'pizza', 'and', 'pasta']
Sentence 5: ['mamma', 'made', 'marmalade']
Sentence 6: ['the', 'quick', 'fox', 'likes', 'fruit']


In [13]:
# the following codes are the answer of Q3.b
# Q3(b) Co-occurrence matrix
import numpy as np
from collections import defaultdict

vocab = sorted(set(word for sent in tokenized_corpus for word in sent))
word_to_id = {w: i for i, w in enumerate(vocab)}
id_to_word = {i: w for w, i in word_to_id.items()}

window_size = 2
V = len(vocab)
co_matrix = np.zeros((V, V), dtype=np.int32)

#  Co-occurrence matrix
for sentence in tokenized_corpus:
    for idx, word in enumerate(sentence):
        w_id = word_to_id[word]
        start = max(0, idx - window_size)
        end = min(len(sentence), idx + window_size + 1)
        for j in range(start, end):
            if j != idx:
                c_id = word_to_id[sentence[j]]
                co_matrix[w_id, c_id] += 1
                co_matrix[c_id, w_id] += 1  

import pandas as pd
co_df = pd.DataFrame(co_matrix, index=vocab, columns=vocab)
display(co_df)

,a,and,arrow,banana,bird,flies,fox,fruit,i,like,...,mamma,marmalade,math,pasta,pizza,quick,slp,swift,the,time
a,0,0,2,2,2,6,0,2,0,6,...,0,0,0,0,0,0,0,2,0,0
and,0,0,0,0,0,0,0,0,0,4,...,0,0,2,2,2,0,2,0,0,0
arrow,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
banana,2,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
bird,2,0,0,0,0,2,0,2,0,2,...,0,0,0,0,0,0,0,0,0,0
flies,6,0,0,0,2,0,0,4,0,6,...,0,0,0,0,0,0,0,0,2,2
fox,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,2,0,0,2,0
fruit,2,0,0,0,2,4,2,0,0,4,...,0,0,0,0,0,0,0,0,2,0
i,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,2,0,2,0,0,0
like,6,4,0,2,2,6,0,4,4,0,...,0,0,0,0,2,0,2,2,0,2


In [35]:
# the following codes are the answer of Q3.c
# Q3(c) PPMI
total_count = np.sum(co_matrix)
sum_w = np.sum(co_matrix, axis=1) 

ppmi_matrix = np.zeros_like(co_matrix, dtype=float)

for i in range(V):
    for j in range(i):
        if co_matrix[i, j] > 0:
            p_wc = co_matrix[i, j] / total_count
            p_w = sum_w[i] / total_count
            p_c = sum_w[j] / total_count
            pmi = np.log2(p_wc / (p_w * p_c))
            ppmi_matrix[i, j] = max(0, pmi)
            ppmi_matrix[j, i] = max(0, pmi)

ppmi_df = pd.DataFrame(ppmi_matrix, index=vocab, columns=vocab)
display(ppmi_df.round(3))

,a,and,arrow,banana,bird,flies,fox,fruit,i,like,...,mamma,marmalade,math,pasta,pizza,quick,slp,swift,the,time
a,0.000,0.000,2.241,2.241,1.241,1.367,0.000,0.071,0.000,0.656,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,1.656,0.000,0.000
and,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.946,...,0.0,0.0,3.115,3.115,2.115,0.000,2.115,0.000,0.000,0.000
arrow,2.241,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,4.115,0.000,0.000
banana,2.241,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.531,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
bird,1.241,0.000,0.000,0.000,0.000,1.241,0.000,1.531,0.000,0.531,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
flies,1.367,0.000,0.000,0.000,1.241,0.000,0.000,1.071,0.000,0.656,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,1.241,2.241
fox,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.531,0.000,0.000,...,0.0,0.0,0.000,0.000,0.000,3.115,0.000,0.000,2.700,0.000
fruit,0.071,0.000,0.000,0.000,1.531,1.071,1.531,0.000,0.000,0.361,...,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,1.531,0.000
i,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.531,...,0.0,0.0,0.000,0.000,2.700,0.000,2.700,0.000,0.000,0.000
like,0.656,0.946,0.000,1.531,0.531,0.656,0.000,0.361,1.531,0.000,...,0.0,0.0,0.000,0.000,0.531,0.000,0.531,0.946,0.000,1.531


In [37]:
# the following codes are the answer of Q3.d
# Q3(d) Cosine similarity function
from numpy.linalg import norm

def cosine_similarity(x, y):
    if norm(x) == 0 or norm(y) == 0:
        return 0.0
    return np.dot(x, y) / (norm(x) * norm(y))

In [39]:
# the following codes are the answer of Q3.e
# Q3(e) Query: top-5 nearest neighbors for "flies"
query = "flies"
query_id = word_to_id[query]
query_vec = ppmi_matrix[query_id]

similarities = {}
for i, w in enumerate(vocab):
    if w != query:
        sim = cosine_similarity(query_vec, ppmi_matrix[i])
        similarities[w] = sim

# 排序：相似度降序，若相同则按字母序
sorted_neighbors = sorted(similarities.items(), key=lambda x: (-x[1], x[0]))

print(f"Top-5 nearest neighbors for '{query}':")
for w, s in sorted_neighbors[:5]:
    print(f"{w:10s}  {s:.4f}")

Top-5 nearest neighbors for 'flies':
like        0.4843
bird        0.4538
banana      0.4410
fruit       0.3497
fox         0.2725


In [49]:
# the following codes are the answer of Q4.a
# Build bigram LM with Laplace smoothing
import re
from collections import defaultdict, Counter

# TRAIN and EVAL strings
train_text = "We carried coffee in reusable cups and we walked along the river path while we reviewed notes."
eval_text  = "Coffee walked and reviewed notes."

train_tokens = re.findall(r"[a-zA-Z]+", train_text.lower())
# Add sentence boundaries around the whole TRAIN sentence
train_seq = ["<s>"] + train_tokens + ["</s>"]

# Tokenize eval and add boundaries
eval_tokens = re.findall(r"[a-zA-Z]+", eval_text.lower())
eval_seq = ["<s>"] + eval_tokens + ["</s>"]

# Vocabulary: all distinct tokens observed in TRAIN (we include <s>, </s>)
vocab = sorted(set(train_seq))
V = len(vocab)

# Mappings
word_to_id = {w:i for i,w in enumerate(vocab)}
id_to_word = {i:w for w,i in word_to_id.items()}

# Bigram counts (count(a,b) where a->b)
bigram_counts = defaultdict(int)
unigram_counts = defaultdict(int)  # count(a) (number of times a occurs as history)

for i in range(len(train_seq)-1):
    a = train_seq[i]
    b = train_seq[i+1]
    bigram_counts[(a,b)] += 1
    unigram_counts[a] += 1

print(vocab)
for (a,b),c in sorted(bigram_counts.items()):
    print(f"{a:6s} -> {b:10s} : {c}")

['</s>', '<s>', 'along', 'and', 'carried', 'coffee', 'cups', 'in', 'notes', 'path', 'reusable', 'reviewed', 'river', 'the', 'walked', 'we', 'while']
<s>    -> we         : 1
along  -> the        : 1
and    -> we         : 1
carried -> coffee     : 1
coffee -> in         : 1
cups   -> and        : 1
in     -> reusable   : 1
notes  -> </s>       : 1
path   -> while      : 1
reusable -> cups       : 1
reviewed -> notes      : 1
river  -> path       : 1
the    -> river      : 1
walked -> along      : 1
we     -> carried    : 1
we     -> reviewed   : 1
we     -> walked     : 1
while  -> we         : 1


In [51]:
# the following codes are the answer of Q4.b
# For every context token that appears in TRAIN (excluding </s>), list top-5 next words under Laplace smoothing.
import math

def laplace_prob(a, b):
    # P(b | a) = (count(a,b) + 1) / (count(a) + V)
    ca = unigram_counts.get(a, 0)
    cab = bigram_counts.get((a,b), 0)
    return (cab + 1) / (ca + V)

# excluding '</s>'
contexts = [w for w in vocab if w != "</s>"]

results = {}
for a in contexts:
    probs = []
    for b in vocab:
        p = laplace_prob(a, b)
        probs.append((b, p))
    probs_sorted = sorted(probs, key=lambda x: (-x[1], x[0]))
    print(f"Context = {a}")
    for b, p in probs_sorted[:topk]:
        print(f"  {b:10s}  {p:.3f}")
    print()


Context = <s>
  we          0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = along
  the         0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = and
  we          0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = carried
  coffee      0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = coffee
  in          0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = cups
  and         0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  carried     0.056

Context = in
  reusable    0.111
  </s>        0.056
  <s>         0.056
  along       0.056
  and         0.056

Context = notes
  </s>        0.111
  <s>         0.056
  along       0.056
  and         0.056
  carried     0.056

Context = path
  while       0.111
  </s>        0.056
  <s>         

In [53]:
# the following codes are the answer of Q4.c
# Sentence probability for eval_seq
from math import prod

prob = 1
for i in range(len(eval_seq)-1):
    a = eval_seq[i]
    b = eval_seq[i+1]
    prob *= laplace_prob(a,b)
    
print(f"P(eval) = {prob:.12e}")

P(eval) = 1.176047764474e-07


In [55]:
# the following codes are the answer of Q4.d
# Perplexity
N = len(eval_seq) - 1 
print(f"Perplexity = {prob ** (-1 / N):.6f}")

Perplexity = 14.286609
